## Dosegljive točke

Tule je nekaj rešitev naloge, ki za podani zemljevid (slovar, katerega ključi so povezave (podane kot pari točk), vrednosti pa veščine, potrebne za uporabo te povezave), začetno točko in veščine, ki jih obvladamo, vrne množico vseh dosegljivih točk.

V nekateri rešitvah bomo uporabljali funkcijo `kam(zemljevid, tocka, vescine)`, ki smo jo sprogramirali v rednem delu domače naloge in pove, kam lahko gremo (neposredno) iz podane točke pri podanem zemljevidu in veščinah.

V osnovi gre za problem iskanja. Temo boste podrobno premlevali v drugem letniku, kjer je boste rekli iskanje v globino oz. širino (depth-first search (DFS), breadth-first search (BFS)).

### Iskanje v širino (in globino), z vrsto (in skladom)

V najpreprostejši rešitvi vzdržujemo vrsto, seznam točk, ki jih moramo še preveriti. V začetku je v seznamu prva točka. Z začetka seznama jemljemo posamične točke in vanj dodajamo vse točke, dosegljive iz te točke, ki jih še nismo pogledali.

In [1]:
def dosegljive(zemljevid, tocka, vescine):
    reachable = set()
    to_check = [tocka]
    while to_check:
        t = to_check.pop(0)
        if t not in reachable and t not in to_check:
            reachable.add(t)
            to_check += kam(zemljevid, t, vescine)
    return reachable

Program ni najbolj učinkovit. Ko vzame točke z začetka seznama (`to_check.pop()`), mora premakniti vse ostale čakajoče točke za eno mesto. To je lahko popraviti: namesto `pop(0)` pokličemo `pop()`, pa bo vzel točko s konca.

In [2]:
def dosegljive(zemljevid, tocka, vescine):
    reachable = set()
    to_check = [tocka]
    while to_check:
        t = to_check.pop()
        if t not in reachable and t not in to_check:
            reachable.add(t)
            to_check += kam(zemljevid, t, vescine)
    return reachable

Prva funkcija išče v širino, druga v globino. Če ne veste, kaj to pomeni, počakajte do drugega letnika.

Brez škode lahko izpustimo drugi pogoj, `t not in to_check`. Če se bo ista točka pojavila v vrsti večkrat, jo bomo obravnavali itak le prvič. Program bo s tem le hitrejši (sploh, če točke jemljemo s konca).

In [3]:
def dosegljive(zemljevid, tocka, vescine):
    reachable = set()
    to_check = [tocka]
    while to_check:
        t = to_check.pop()
        if t not in reachable:
            reachable.add(t)
            to_check += kam(zemljevid, t, vescine)
    return reachable

Narediti smemo tudi kar takole.

In [8]:
def dosegljive(zemljevid, tocka, vescine):
    reachable = set()
    to_check = [tocka]
    for t in to_check:
        if t not in reachable:
            reachable.add(t)
            to_check += kam(zemljevid, t, vescine)
    return reachable

Tule gre spet iskanje v širino, z vrsto. Razlika je v tem, da elementov ne brišemo, temveč se le pomikamo naprej po vrsti. In, kar je še bolj imenitno, to počnemo kar s `for`. Če v seznam dodajamo nove elemente, `for` tega "ne opazi" -- v smislu, ne vidi, da so bili dodani šele naknadno in i

### Iskanje v valovih

Nalogo lahko rešimo tako, da najprej dodamo vse sosede začetne točke. Nato dodamo sosede teh sosedov (izvzemši tiste, za katere že vemo). Potem njihove sosede ... in tako naprej, dokler ni več nepregledanih sosedov.

In [4]:
def dosegljive(zemljevid, tocka, vescine):
    reachable = set()
    to_check = {tocka}
    while to_check:
        reachable |= to_check
        new_check = set()
        for point in to_check:
            new_check |= kam(zemljevid, point, vescine)
        to_check = new_check - reachable
    return reachable

### Rekurzivna rešitev z akumulatorjem

Tale rekurzivna rešitev je praktično enaka rešitvi s skladom (torej tisti s seznamom in `pop()`), le zanko smo nadomestili z rekurzijo.

Zanimiva je zato, ker smo pomožno funkcijo napisali kot lokalno funkcijo funkcije `dosegljive`. S tem dosežemo, da nam ni treba podajati argumentov, ki se med rekurzijo ne spreminjajo. Pa še "globalnega" imenskega prostora nismo okužili s funkcijo, ki jo potrebujemo le v v funkciji `dosegljive`.

Ne spreglejte tega: funkcija spreminja množico `reachable`. Zato ni nobene potrebe, da bi jo vračala. Prav tako ni potrebe, da bi jo pomožni funkciji podajali kot argument, saj jo tako ali tako vidi.

In [5]:
def dosegljive(zemljevid, tocka, vescine):
    reachable = set()

    def _dosegljive(t):
        if t not in reachable:
            reachable.add(t)
            for t2 in kam(zemljevid, t, vescine):
                _dosegljive(t2)

    _dosegljive(tocka)
    return reachable

Za študente bolj običajna različica istega je takšna.

In [6]:
def dosegljive(zemljevid, tocka, vescine):
    reachable = {tocka}
    _dosegljive(zemljevid, tocka, vescine, reachable)
    return reachable

def _dosegljive(zemljevid, point, skills, reachable):
    for dest in kam(zemljevid, point, skills):
        if dest not in reachable:
            reachable.add(dest)
            _dosegljive(zemljevid, dest, skills, reachable)

Razlika je v tem, da moramo zdaj podajati originalne argumente (`zemljevid`, `tocka`, `vescine`), pa tudi `reachable`. Spet pa: funkcija `_dosegljive` ne vrača `reachable`, ker ta rezultat nikogar ne zanima. V rekurzivnem klicu je nepotreben, v `dosegljive` pa imamo `reachable` tako ali tako na voljo.

### Rekurzivna rešitev z rezultatom

Osebno so mi vedno všečnejše rekurzine funkcije, ki rezultat vrnejo z `return`, ne z argumentom. Osebna preferenca. V praksi so takšne, ki vračajo z argumentom *lahko* hitrejše, ker jih znajo nekateri jeziki na določen način optimirati, če jih napišemo na določen način ([tail call](https://en.wikipedia.org/wiki/Tail_call). Python ni eden od teh jezikov.

Meni se zdi (spet: oseben okus!) najbolj elegantna takšna rešitev.

In [7]:
def dosegljive(zemljevid, tocka, vescine):
    paths = defaultdict(dict)
    for (src, dest), required in zemljevid.items():
        paths[src][dest] = required

    return _dosegljive(paths, tocka, vescine)

def _dosegljive(paths, point, skills):
    reachable = {point}
    where = paths.pop(point, {})
    for dst, required in where.items():
        if required <= skills:
            reachable |= _dosegljive(paths, dst, skills)
    return reachable

`zemljevid` predelamo v slovar, katerega ključi so začetne točke povezav, vrednosti pa slovarji, katerih ključi so končne točke, pripadajoče vrednosti pa zahtevane veščine. (Če ne razumete `defaultdict(dict)`, nič hudega. Glejte le zanko, ki sledi.) Tak slovar je prikladnejši, poleg tega pa s tem mimogrede naredimo kopijo.

Glavni trik te rešitve je, da iz slovarja (`paths`) odstranimo vsako obiskano točko in s tem preprečimo, da bi kdaj ponovno preverjali povezave, ki vodijo iz nje.

Čistuni bi se pritožili, da ta funkcija spreminja vrednost argumenta `paths`. Da, namesto tega bi lahko sestavili nov `paths`, ki bi vseboval celoten slovar `paths` razen tega elementa. To bi šlo, vendar bi postala funkcija manj učinkovita (ker bi lahko prišla do iste točke na različne načine in jo nato ponovno obravnavala), pa tudi manj elegantno bi bilo.